In [1]:
import os

In [2]:
%pwd

'c:\\Resume Projects\\Kidney-Tumor-Classifier\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Resume Projects\\Kidney-Tumor-Classifier'

# Entity

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

# Configuration Manager

In [6]:
from KidneyTumorClassifier.constants import *
from KidneyTumorClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(root_dir=config.root_dir,source_URL=config.source_URL,
                                                    local_data_file=config.local_data_file,unzip_dir=config.unzip_dir)
        return data_ingestion_config


# Update Components

In [8]:
import os
import zipfile
import gdown
from KidneyTumorClassifier import logger
from KidneyTumorClassifier.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self) -> str:
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")
            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id, zip_download_dir)
            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")
        except Exception as e:
            raise e
        
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
    

# Create Pipeline

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e
    

[2024-09-21 14:07:48,925: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-09-21 14:07:48,932: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-21 14:07:48,938: INFO: common: Created directory at: artifacts]
[2024-09-21 14:07:48,944: INFO: common: Created directory at: artifacts/data_ingestion]
[2024-09-21 14:07:48,949: INFO: 4218606990: Downloading data from https://drive.google.com/file/d/1-xFCNBMXSfdRr_UpHkc62hiCG1rKMMjm/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1-xFCNBMXSfdRr_UpHkc62hiCG1rKMMjm
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1-xFCNBMXSfdRr_UpHkc62hiCG1rKMMjm&confirm=t&uuid=0b0594d7-83d3-4139-ad3a-ac7854b15271
To: c:\Resume Projects\Kidney-Tumor-Classifier\artifacts\data_ingestion\data.zip
100%|██████████| 57.7M/57.7M [00:11<00:00, 5.08MB/s]

[2024-09-21 14:08:07,236: INFO: 4218606990: Downloaded data from https://drive.google.com/file/d/1-xFCNBMXSfdRr_UpHkc62hiCG1rKMMjm/view?usp=sharing into file artifacts/data_ingestion/data.zip]
